In [1]:
import PyPDF2
from pdf2image import convert_from_path
import pytesseract
import numpy as np #pip install PyPDF2 pdf2image pytesseract numpy
import logging
from textsplitter import TextSplitter #pip install textsplitter
from langchain.text_splitter import CharacterTextSplitter #pip install langchain
#from openai.embeddings_utils import get_embedding
import re
from sentence_transformers import SentenceTransformer#pip install sentence_transformers
import faiss
import os
from openai import OpenAI
import openai
import numpy as np


In [2]:
def extract_sorted_entities(file_path): #output 1 dim string list
    # 读取指定路径的UTF-8编码的文件
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    # 使用 || 分割实体
    entities = data.split('||')

    # 去除每个实体两端的空格，并删除空字符串
    entities = [entity.strip() for entity in entities if entity.strip()]

    # 去除重复的实体
    unique_entities = set(entities)

    # 对实体进行排序
    sorted_entities = sorted(unique_entities)

    return sorted_entities#1d string list


In [3]:
entities_path = r"C:\Users\Li\Desktop\output_entities.txt"
entities = extract_sorted_entities(entities_path)
print(entities) #1 dim string list

['"First principles methods using CASTEP", Zeitschrift fuer Kristallographie 220(5-6) pp. 567-570 (2005) S. J. Clark, M. D. Segall, C. J. Pickard, P. J. Hasnip, M. J. Probert, K. Refson, M. C. Payne', '#', '#documentation', '#gsc.tab=0', "'is compatible license of' provides a method of marking two software licenses as compatible and without conflicts, e.g. that the Apache License version 2 is compatible with GNU GPL version 3. If two licenses are connected with this property, it means code released under one license can be released with code from the other license in a larger program.", '(forall (x) (if (Role x) (RealizableEntity x))) // axiom label in BFO2 CLIF: [061-001]', '(optionally) LAMMPS', '-', '.eln', '.h5', '.ipynb', '.json', '.owl', '0.0.4', '0.0.6', '0.10.0', '0.10.1', '0.10.9', '0.2.7', '0.26.0', '0.3', '0.4', '0.4.0', '0.4.6', '0.5', '0.6', '0.7.0', '0.7.6', '0.8.9', '0000-0001-6714-0954', '0000-0001-6884-0530', '0000-0001-7069-9804', '0000-0001-7192-7143', '0000-0001-743

In [4]:
def read_questions(file_path):#output 1 dim string list
    # 读取指定路径的UTF-8编码的文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 使用换行符分割文件内容，得到问题列表
        questions = file.read().split('\n')
    
    # 去除每个问题两端的空格，并删除空字符串
    questions = [question.strip() for question in questions if question.strip()]
    
    return questions

# 调用函数并打印结果
questions = read_questions(r"C:\Users\Li\Desktop\questions.txt")
print(questions)


['Who is working in the Computational Materials Science field?', 'What are the research projects associated to EMMO?', 'Who are the contributors of the data "datasets"?', 'Who is working with Researcher "Ebrahim Norouzi" in the same group?', 'who is the email address of "ParaView"?', 'What are the affilliations of Volker Hofmann?', 'What is "Molecular Dynamics" Software? List the programming language, documentation page, repository, and license information.', 'What are pre- and post-processing tools for MD simulations?', 'What are some workflow environments for computational materials science?', 'How should I cite pyiron?', 'Where can I find a list of interatomic potentials?', 'What are python libraries used for calculating local atomic structural environment?', 'What are the electronic lab notebooks available?', 'What are the software for Molecular Dynamics (MD)?', 'What are the ontologies in nanomaterials domain?', 'What is DAMASK?', 'What are the data portals for materials science o

In [5]:
def get_data_max_chunk_size_chunks(data: str, max_chunk_size: int):#retunn 2d string list
    # 使用正则表达式匹配所有的 triples
    pattern = r'<-(.*?)->'
    triples = re.findall(pattern, data, re.DOTALL)

    # 按第一个单词排序
    sorted_triples = sorted(triples, key=lambda x: x.split()[0] if x.strip() else '')

    # 将排序后的 triples 分块
    chunks = []
    current_chunk = []
    current_word = None

    for triple in sorted_triples:
        first_word = triple.split()[0] if triple.strip() else ''
        if first_word != current_word or len(current_chunk) >= max_chunk_size:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = [triple]
            current_word = first_word
        else:
            current_chunk.append(triple)

    # 添加最后一个 chunk
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

In [6]:
file_path = r'C:\Users\Li\Desktop\modified_op.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()
data
#unprocessed data

'<-Web Standards and Reference Data for First-Principles Simulations has label Web Standards and Reference Data for First-Principles Simulations->\n\n<-BAM reference data: results of ASTM E139 -11 creep tests on a reference material of Nimonic 75 nickel-base alloy object Type is test data->\n\n<-Elastic Constant Demo SWO_0000082 Jupyter lab->\n\n<-info@quantum-espresso.org email Address is info@quantum-espresso.org->\n\n<-GPL 3 has label GPL 3->\n\n<-http://www.bio.uni-freiburg.de/fakultaet/institute/bio2 22-rdf-syntax-ns#type website->\n\n<-E914277 22-rdf-syntax-ns#type publication->\n\n<-FAIRmat 22-rdf-syntax-ns#type funding project->\n\n<-https://www.yambo-code.eu/ url.n.01 ->\n\n<-http://www.qhull.org/ 22-rdf-syntax-ns#type website->\n\n<-Crystallographic Information Framework (CIF2) citation.n.01 Bernstein, H. J., Bollinger, J. C., Brown, I. D., Gražulis, S., Hester, J. R., McMahon, B., Spadaccini, N., Westbrook, J. D. & Westrip, S. P. (2016). "Specification of the Crystallographi

In [7]:
#transfer data into chunks
chunks = get_data_max_chunk_size_chunks(data,30)#max averages of char.

i=1
for chunk in chunks:
    print(i,".#######################################")
    i=i+1
    print(chunk)

1 .#######################################
['(optionally) LAMMPS 22-rdf-syntax-ns#type resource', '(optionally) LAMMPS has label (optionally) LAMMPS']
2 .#######################################
['.ipynb 22-rdf-syntax-ns#type File Format', '.ipynb has label .ipynb']
3 .#######################################
['.json has label .json', '.json 22-rdf-syntax-ns#type File Format']
4 .#######################################
['.owl has label .owl', '.owl 22-rdf-syntax-ns#type File Format']
5 .#######################################
['1. Saal, J. E., Kirklin, S., Aykol, M., Meredig, B., and Wolverton, C. Materials Design and Discovery with High-Throughput Density Functional Theory: The Open Quantum Materials Database (OQMD), JOM 65, 1501-1509 (2013). doi:10.1007/s11837-013-0755-4\n\n2. Kirklin, S., Saal, J.E., Meredig, B., Thompson, A., Doak, J.W., Aykol, M., Rühl, S. and Wolverton, C. "The Open Quantum Materials Database (OQMD): assessing the accuracy of DFT formation energies", npj Computatio

In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def extract_queries(entities, questions):
    # 初始化SentenceTransformer模型
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # 计算实体的嵌入向量
    entity_embeddings = model.encode(entities, convert_to_tensor=True)

    # 创建faiss索引
    index = faiss.IndexFlatL2(entity_embeddings.shape[1])
    index.add(np.array(entity_embeddings))

    # 计算问题的嵌入向量
    question_embeddings = model.encode(questions, convert_to_tensor=True)

    # 初始化结果列表
    queries = []

    # 对每个问题，找到最相关的实体
    for question_embedding in question_embeddings:
        # 搜索最近的2个实体
        distances, indices = index.search(np.array(question_embedding).reshape(1, -1), 2)

        # 选取相关实体，并确保不重复
        related_entities_set = set()
        related_entities = []
        for i in indices[0]:
            if entities[i] not in related_entities_set:
                related_entities.append(entities[i])
                related_entities_set.add(entities[i])

        # 添加到结果中
        queries.append(related_entities)

    return queries


In [9]:
queries = extract_queries(entities, questions)

for i, query in enumerate(queries):
    print(f"Question {i+1}: {questions[i]}")
    print(f"Related entities: {query}\n")

Question 1: Who is working in the Computational Materials Science field?
Related entities: ['Computational Material Science', 'Computational Materials Science']

Question 2: What are the research projects associated to EMMO?
Related entities: ['EMMO', 'https://github.com/emmo-repo/EMMO']

Question 3: Who are the contributors of the data "datasets"?
Related entities: ['datasets', 'A relation between a dataset and a person or organization, who has created the data.']

Question 4: Who is working with Researcher "Ebrahim Norouzi" in the same group?
Related entities: ['Ebrahim Norouzi', 'Norouzi']

Question 5: who is the email address of "ParaView"?
Related entities: ['ParaView', 'paraview']

Question 6: What are the affilliations of Volker Hofmann?
Related entities: ['Dr.  Volker Hofmann', 'Hofmann']

Question 7: What is "Molecular Dynamics" Software? List the programming language, documentation page, repository, and license information.
Related entities: ['molecular modelling and visualiz

In [10]:
print(chunks)

[['(optionally) LAMMPS 22-rdf-syntax-ns#type resource', '(optionally) LAMMPS has label (optionally) LAMMPS'], ['.ipynb 22-rdf-syntax-ns#type File Format', '.ipynb has label .ipynb'], ['.json has label .json', '.json 22-rdf-syntax-ns#type File Format'], ['.owl has label .owl', '.owl 22-rdf-syntax-ns#type File Format'], ['1. Saal, J. E., Kirklin, S., Aykol, M., Meredig, B., and Wolverton, C. Materials Design and Discovery with High-Throughput Density Functional Theory: The Open Quantum Materials Database (OQMD), JOM 65, 1501-1509 (2013). doi:10.1007/s11837-013-0755-4\n\n2. Kirklin, S., Saal, J.E., Meredig, B., Thompson, A., Doak, J.W., Aykol, M., Rühl, S. and Wolverton, C. "The Open Quantum Materials Database (OQMD): assessing the accuracy of DFT formation energies", npj Computational Materials 1, 15010 (2015). doi:10.1038/npjcompumats.2015.10 Link\n(See a list of citing articles here). "http://www.w3.org/2000/01/rdf-schema#label" 1. Saal, J. E., Kirklin, S., Aykol, M., Meredig, B., and 

In [11]:
def match_queries_with_chunks(queries, chunks):
    # 初始化SentenceTransformer模型
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # 将每个chunk的triples合并成一个字符串，并计算嵌入向量
    chunk_strings = [' '.join(chunk) for chunk in chunks]
    chunk_embeddings = model.encode(chunk_strings, convert_to_tensor=True)

    # 创建faiss索引
    index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
    index.add(np.array(chunk_embeddings))

    # 初始化结果列表
    matched_chunks = []

    # 对每个query，找到最相关的两个chunks
    for query in queries:
        # 计算query的嵌入向量
        query_embedding = model.encode([query], convert_to_tensor=True)
        
        # 搜索最相关的两个chunks
        distances, indices = index.search(np.array(query_embedding), 2)

        # 选取相关chunks
        related_chunks = [chunks[i] for i in indices[0]]

        # 添加到结果中
        matched_chunks.append(related_chunks)

    return matched_chunks

In [12]:
matched_chunks = match_queries_with_chunks(queries, chunks)



In [24]:
# 打印前10个问题的相关块
#print(matched_chunks)
print("1111111111111111111111")
for i, related_chunks in enumerate(matched_chunks[:]):
    print(f"Question {i+1}:",questions[i])
    print(f"Related chunks: {related_chunks}\n")
    #context= ' '.join(related_chunks)
    final_string = ""

    # 遍历二维列表
    for inner_list in related_chunks:
        for item in inner_list:
            # 将每个元素拼接到final_string，并在元素间添加空格
            final_string += item + " "
        # 在每个子列表的最后添加句号
        final_string = final_string.strip() + ". "
    
    # 移除最终字符串的尾随空格
    final_string = final_string.strip()
    
    print(final_string)

1111111111111111111111
Question 1: Who is working in the Computational Materials Science field?
Related chunks: [['MaterialsProject citation.n.01 A. Jain*, S.P. Ong*, G. Hautier, W. Chen, W.D. Richards, S. Dacek, S. Cholia, D. Gunter, D. Skinner, G. Ceder, K.A. Persson (*=equal contributions)\nThe Materials Project: A materials genome approach to accelerating materials innovation\nAPL Materials, 2013, 1(1), 011002.\ndoi:10.1063/1.4812323', 'MaterialsProject has Website https://materialsproject.org/', 'MaterialsProject conforms To Specification OPTIMADE', 'MaterialsProject has license Creative Commons Attribution 4.0 International License', 'MaterialsProject has RelatedResource custodian', 'MaterialsProject description.n.01 By computing properties of all known materials, the Materials Project aims to remove guesswork from materials design in a variety of applications. Experimental research can be targeted to the most promising compounds from computational data sets. Researchers will be 

In [35]:
def get_answers_with_chunks(questions, matched_chunks):
    answers = []
    api_key = os.environ.get('OPENAI_API_KEY')  
    openai.api_key = api_key
    client = OpenAI(api_key=api_key)
    model="gpt-3.5-turbo-1106"
    for i, related_chunks in enumerate(matched_chunks[:]):
        print(f"Question {i+1}:", questions[i])
        print(f"Related chunks: {related_chunks}\n")

        # 将二维列表中的元素拼接成一个字符串
        final_string = ""
        for inner_list in related_chunks:
            for item in inner_list:
                final_string += item + " "
            final_string = final_string.strip() + ". "
        final_string = final_string.strip()

        try:
            # 发送请求到GPT-3.5
            messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": final_string + questions[i]}
            ]
            response = client.chat.completions.create(
                model=model,
                messages=messages
            )

            # 提取并存储答案
            answer = response.choices[0].message.content
            print("Answer:", answer)
            answers.append(answer)

        except Exception as e:
            print(f"处理第 {i+1} 个问题时发生错误: {e}")
            answers.append("无法获取答案")

        print(final_string)

    return answers

In [36]:
answers = get_answers_with_chunks(questions,matched_chunks)

Question 1: Who is working in the Computational Materials Science field?
Related chunks: [['MaterialsProject citation.n.01 A. Jain*, S.P. Ong*, G. Hautier, W. Chen, W.D. Richards, S. Dacek, S. Cholia, D. Gunter, D. Skinner, G. Ceder, K.A. Persson (*=equal contributions)\nThe Materials Project: A materials genome approach to accelerating materials innovation\nAPL Materials, 2013, 1(1), 011002.\ndoi:10.1063/1.4812323', 'MaterialsProject has Website https://materialsproject.org/', 'MaterialsProject conforms To Specification OPTIMADE', 'MaterialsProject has license Creative Commons Attribution 4.0 International License', 'MaterialsProject has RelatedResource custodian', 'MaterialsProject description.n.01 By computing properties of all known materials, the Materials Project aims to remove guesswork from materials design in a variety of applications. Experimental research can be targeted to the most promising compounds from computational data sets. Researchers will be able to data-mine scien

In [37]:
def save_answers_to_file(answers, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for answer in answers:
            file.write(answer + "\n\n")
file_path = r'C:\Users\Li\Desktop\answer.txt'
save_answers_to_file(answers, file_path)

In [48]:
api_key = os.environ.get('OPENAI_API_KEY')
openai.api_key = api_key
client = OpenAI(api_key=api_key)

def get_answers(questions):
    answers = []
    i=1
    for question in questions:
        try:
            # 发送请求到GPT-3.5
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": question}
                ]
            )

            # 提取并存储答案
            answer = response.choices[0].message.content
            print("Answer No.",i,":")
            print(answer)
            answers.append(answer)
            i=i+1
        except Exception as e:
            print(f"处理问题时发生错误: {e}")
            answers.append("无法获取答案")

    return answers

In [49]:
answers = get_answers(questions)

Answer No. 1 :
There are many researchers and scientists working in the field of Computational Materials Science. Some well-known individuals in the field include:

1. Gerbrand Ceder: A professor of Materials Science, Ceder is known for his work on high-throughput computational materials design and the development of materials for energy storage.

2. Nicola Marzari: Marzari is a professor of Materials Science and Engineering, known for his work on computational materials design and the development of new materials for applications in electronics and energy.

3. Stefano Curtarolo: Curtarolo is a professor of Mechanical Engineering and Materials Science and a pioneer in the use of computational methods to discover and design new materials.

4. Richard Needs: A professor of Materials Science at the University of Cambridge, Needs is known for his work on the development and application of quantum mechanical methods in materials science.

5. Sharon Glotzer: Glotzer is a professor of Chemica

In [40]:
def save_answers_to_file(answers, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for answer in answers:
            file.write(answer + "\n\n")
file_path = r'C:\Users\Li\Desktop\answer1.txt'
save_answers_to_file(answers, file_path)